<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Training5/blob/main/translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#install package
!pip install opencc

     |████████████████████████████████| 765 kB 8.3 MB/s 


In [3]:
from google.colab import drive
drive.mount("gdrive")

Mounted at gdrive


In [4]:
!unzip /content/gdrive/MyDrive/translation2019zh.zip

Archive:  /content/gdrive/MyDrive/translation2019zh.zip
  inflating: translation2019zh_train.json  
  inflating: translation2019zh_valid.json  


# read json

In [6]:
import json
import opencc
train_data = []

x=1
chineseCc = opencc.OpenCC('s2hk.json')
with open("translation2019zh_train.json",newline="") as js:
  for i in js:
    data = json.loads(i)
    if data["english"].isascii() == False:
      continue
    if len(data["english"].strip()) > 25:
      continue
    if len(data["chinese"].strip()) > 8:
      continue
    data["english"] = data["english"].replace("\\","").strip()
    data["chinese"] = chineseCc.convert(data["chinese"]).strip()
    train_data.append(data)
    if(x==20000):
      break
    x+=1
  js.close()

In [7]:
txt_list = []

for data in train_data:
  strs = "%s\t%s"%(data["english"],data["chinese"])
  txt_list.append("%s\t%s"%(data["english"],data["chinese"]))

In [8]:
#wirte file to txt
with open("engChinese.txt","w",encoding="utf-8") as f:
  for data in txt_list:
    f.write(data)
    f.write("\n")

# Creating the model :Seq2Seq

In [9]:
from keras.models import Model
from keras.layers import Input, LSTM,Dense
import numpy as np

BATCH_SIZE = 64
EPOCHS = 200
LATENT_DIM = 256
NUM_SAMPLES = 10000
#traning setting

file_path = "engChinese.txt"

In [10]:
#Vetorize data
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

with open(file_path,"r",encoding="utf-8") as f:
  lines = f.read().split("\n") #each line is split by \n
  for line in lines[:min(NUM_SAMPLES,len(lines)-1)]: #num_spaces or line is larger,else will out of range
    input_text, target_text = line.split("\t") #left side of \t is end ,right side is chinese
    
    #each chinese texts need to add \t at the begining and \n at the end of the text
    target_text = "\t" + target_text +"\n"

    #add to input and target list
    input_texts.append(input_text)
    target_texts.append(target_text)

    #loopping the text to get all chars and make one-hot encoding later
    for char in input_text: #for english
      if char not in input_chars:
        input_chars.add(char)

    for char in target_text: # for chinese
      if char not in target_chars:
        target_chars.add(char)


In [11]:
#handling the data to generate some required info
# sorting the chars 
input_chars = sorted(list(input_chars)) # according to Unicode
target_chars = sorted(list(target_chars)) # according to Unicode

#total chars of input and target
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)

#the maxinums sentance of input and target
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#log out the message for those info
print("Num of sample",len(input_texts))
print("Num of unique input token",num_encoder_tokens)
print("Num of unique output token",num_decoder_tokens)
print("Max seq length of inputs",max_encoder_seq_length)
print("Max seq length of output",max_decoder_seq_length)

Num of sample 10000
Num of unique input token 85
Num of unique output token 3314
Max seq length of inputs 25
Max seq length of output 10


In [14]:
#Creating Encoder-LSTM

#make each chars has an index
input_token_index = dict([(char , i) for i ,char in enumerate(input_chars)]) # english token with index
target_token_index = dict([(char , i) for i ,char in enumerate(target_chars)]) # chinese token with index

#Need 3 array with 3-dimension(num_simple,max_seq_text,total_token)
#Encoder-input,Decoder-input,Decoder-output
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens), #shape-3d
    dtype=np.float32 #type float32
)

decoder_input_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens), #shape-3d
    dtype=np.float32 #type float32
)

decoder_output_data = np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens), #shape-3d
    dtype=np.float32 #type float32
)

#zip eng sentance and chinese sentance(eng[0]Chinese[0],eng[1]Chinese[1]...etc)
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  #print(i,(input_text,target_text))
  for t,char in enumerate(input_text):
    #change the array of input data =>i:x,t:y,token:z to 1 = only set z-index to 1
    encoder_input_data[i,t,input_token_index[char]] = 1
  for t,char in enumerate(target_text):
    #chagne the array of target data
    #decoder_target_data is ahead decoder_input_data by one timestemp
    decoder_input_data[i,t,target_token_index[char]] = 1
    if t > 0:
      #only affect t > 0 character
      decoder_output_data[i,t-1,target_token_index[char]] = 1

# Encoder

In [15]:
#Define input tensor
encoder_inputs = Input(shape=(None,num_encoder_tokens)) #Shape 1*num_encoder_tokens

#encoder LSTM with units = 256 and return the last state
encoder_lstm = LSTM((LATENT_DIM),return_state=True)

#output of encoder lstm tensor,return state history and state current
encoder_outputs,state_h,state_c = encoder_lstm(encoder_inputs)

#keep states only
encoder_states = [state_h,state_c]

# Decoder

In [16]:
#need encoder states as decoder input
#define decoder lstm and tensor
decoder_inputs = Input(shape=(None,num_decoder_tokens)) #shape 1*num_decoder_tokens

#decoder_lstm ,set return everything
decoder_lstm = LSTM((LATENT_DIM),return_state=True,return_sequences=True)

#igore the internal state
decoder_outputs,_,_ = decoder_lstm(decoder_inputs,initial_state=encoder_states)


In [17]:
#creating the Dense for full connected layer
decoder_dense = Dense(num_decoder_tokens,activation="softmax") #num_decoder_tokens * 1

decoder_outputs = decoder_dense(decoder_outputs) #passing lstm output to decoder dense

In [18]:
#Define the model
#input encoder input data and decoder input data and return decoder output data
model = Model([encoder_inputs,decoder_inputs],decoder_outputs)

#Run traning and complie the model
model.compile(optimizer="rmsprop",loss="categorical_crossentropy")

#fit data to model and training
model.fit([encoder_input_data,decoder_input_data],decoder_output_data,batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.2)

#save our model
model.save("seq2seq.h5")

Epoch 1/200
125/125 [==============================] - 12s 32ms/step - loss: 4.4282 - val_loss: 4.3165
Epoch 2/200
125/125 [==============================] - 3s 22ms/step - loss: 4.2697 - val_loss: 4.2947
Epoch 3/200
125/125 [==============================] - 3s 22ms/step - loss: 4.2222 - val_loss: 4.2874
Epoch 4/200
125/125 [==============================] - 3s 22ms/step - loss: 4.1767 - val_loss: 4.2466
Epoch 5/200
125/125 [==============================] - 3s 22ms/step - loss: 4.1235 - val_loss: 4.2101
Epoch 6/200
125/125 [==============================] - 3s 22ms/step - loss: 4.0678 - val_loss: 4.1371
Epoch 7/200
125/125 [==============================] - 3s 22ms/step - loss: 4.0055 - val_loss: 4.1167
Epoch 8/200
125/125 [==============================] - 3s 22ms/step - loss: 3.9353 - val_loss: 4.0568
Epoch 9/200
125/125 [==============================] - 3s 22ms/step - loss: 3.8559 - val_loss: 4.0192
Epoch 10/200
125/125 [==============================] - 3s 22ms/step - loss: 3.77

In [20]:
# Define sampling models
#inferencemode (sampling).
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(LATENT_DIM,))
decoder_state_input_c = Input(shape=(LATENT_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# lstm
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [21]:
#reverse char-index
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [22]:

def decodeSentence(input_seq):
    state = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + state)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # find char using index
        sampled_char = reverse_target_char_index[sampled_token_index]
        # and append sentence
        sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        state = [h, c]

    return sentence


In [24]:
for seq_index in range(100,200):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    print(input_seq)
    decoded_sentence = decodeSentence(input_seq)
    print('-')
    print('English:', input_texts[seq_index])
    print('Translate(Chinese):', decoded_sentence)

[[[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
-
English: I can't look below that.
Translate(Chinese): 我不會再往下看。

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
-
English: Road to the Dan smile.
Translate(Chinese): 于丹微笑道。

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
-
English: Front wheel drive.
Translate(Chinese): 前輪驅動。

[[[0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
-
English: I would fain help you .
Translate(Chinese): 我樂意幫助你。

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0.